# Hotel Finder

In [2]:
# Dependencies and Setup
import hvplot.pandas
import holoviews as hv
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the city data
city_data_df = pd.read_csv("data/city_data.csv")

# Display sample data
city_data_df.head()

,City,Lat,Lng,Max Temp,Country,Date
0,ust-nera,64.5667,143.2000,-25.18,RU,1734587315
1,berriane,32.8265,3.7669,9.53,DZ,1734587316
2,qaqortoq,60.7167,-46.0333,-1.55,GL,1734587317
3,port saint john's,-31.6229,29.5448,21.00,ZA,1734587318
4,martaban,16.5314,97.6111,31.13,MM,1734587319


---

### Map of Cities

### Narrow down the list of cities to fit your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values

# Example Criteria: between 18 and 26 degrees Celsius, less than 60% humidity
city_data_df = city_data_df[(city_data_df["Max Temp"] <=26) & (city_data_df["Max Temp"] >=18)]

# Drop any rows with null values
city_data_df.dropna(how="any")

# Display sample data
city_data_df.head()

,City,Lat,Lng,Max Temp,Country,Date
3,port saint john's,-31.6229,29.5448,21.00,ZA,1734587318
6,wailua homesteads,22.0669,-159.3780,23.44,US,1734587321
7,invercargill,-46.4000,168.3500,18.90,NZ,1734587322
11,zalingei,12.9096,23.4741,22.46,SD,1734587327
13,los arabos,22.7281,-80.7214,21.50,CU,1734587329


In [43]:
# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng", "Lat", geo=True, tiles="OSM", size=25, color="darkslategray",
    xlim=(-180, 180), ylim=(-60, 60), alpha=0.5,
    width=800, height=400
).opts(aspect="equal")

hv.save(map_plot, "visuals/city_map.html")

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]

### New DataFrame called `hotel_df`.

In [16]:
# Create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df.copy()
hotel_df = hotel_df[["City", "Country", "Lat", "Lng"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = [""] * len(hotel_df)

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
3,port saint john's,ZA,-31.6229,29.5448,
6,wailua homesteads,US,22.0669,-159.3780,
7,invercargill,NZ,-46.4000,168.3500,
11,zalingei,SD,12.9096,23.4741,
13,los arabos,CU,22.7281,-80.7214,


### Find the first hotel located within 10,000 metres of the city using Geoapify API

In [17]:
# Set parameters to search for a hotel

radius = 10000

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Parameters
    params = {
        "categories": "accommodation.hotel",
        "limit": 1,
        "filter": f"circle:{longitude},{latitude},{radius}",
        "bias": f"proximity:{longitude},{latitude}",
        "apiKey": geoapify_key
    }

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

hotel_df.to_csv("data/hotels.csv", header=True)

# Display sample data
hotel_df

Starting hotel search
port saint john's - nearest hotel: Outback Inn
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
invercargill - nearest hotel: The Grand
zalingei - nearest hotel: No hotel found
los arabos - nearest hotel: No hotel found
blackmans bay - nearest hotel: Villa Howden
huarmey - nearest hotel: Hostal Santa Rosa
port elizabeth - nearest hotel: Waterford Hotel
byron bay - nearest hotel: Lord Byron Resort
pisco - nearest hotel: La Portada
adamstown - nearest hotel: No hotel found
hermanus - nearest hotel: Aloe guest house
bouca - nearest hotel: No hotel found
mar del plata - nearest hotel: Hotel 3 de Abril
plettenberg bay - nearest hotel: Milkwood Manor
bredasdorp - nearest hotel: Victoria Hotel
holualoa - nearest hotel: Kona Hotel
amarante do maranhao - nearest hotel: No hotel found
la'ie - nearest hotel: No hotel found
veraval - nearest hotel: Shyam
al ghayzah - nearest hotel: فندق تاج العرب
sotik post - nearest hotel: Brevan Hotel
campo mourao - nea

,City,Country,Lat,Lng,Hotel Name
3,port saint john's,ZA,-31.6229,29.5448,Outback Inn
6,wailua homesteads,US,22.0669,-159.3780,Hilton Garden Inn Kauai Wailua Bay
7,invercargill,NZ,-46.4000,168.3500,The Grand
11,zalingei,SD,12.9096,23.4741,No hotel found
13,los arabos,CU,22.7281,-80.7214,No hotel found
...,...,...,...,...,...
562,kingaroy,AU,-26.5333,151.8333,No hotel found
564,anna regina,GY,7.2644,-58.5077,Jaigobin Hotel
567,guajara mirim,BR,-10.7828,-65.3394,Novo Hotel Campos
570,dapa,PH,9.7594,126.0531,Island Suites


### Add hovers to the map

In [44]:
# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    "Lng", "Lat", geo=True, tiles="OSM", size=25, color="darkslategray",
    xlim=(-180, 180), ylim=(-60, 60),
    alpha=0.5, hover_cols=["City", "Hotel Name", "Country"],
    width=800, height=400
).opts()

hv.save(hotel_plot, "visuals/hotel_map.html")

# Display the map
hotel_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)